In [ ]:
expression = "client-ip=[0-9]+(?:\.[0-9]+)+"
search_string = " pass (zohomail.com: domain of gator4222.hostgator.com designates 192.185.50.45 as permitted sender) client-ip=192.185.50.45; envelope-from=vajra@gator4222.hostgator.com; helo=gateway24.websitewelcome.com;"

import re
ip = re.findall(r'client-ip=[0-9]+(?:\.[0-9]+)+', search_string)
print(ip[0].split("=")[1])

In [8]:
# Reference: https://www.thepythoncode.com/article/reading-emails-in-python

import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import re

# account credentials
username="info@vpms.club"
password="n9/ut4C?32P["
imap = imaplib.IMAP4_SSL("imappro.zoho.com")
# authenticate
imap.login(username, password)

status, messages = imap.select("INBOX")
# number of top emails to fetch
N = int(messages[0])
# total number of emails
messages = int(messages[0])

print("Total Messages: ", messages)

print(f'from;user_ip_address;user_email_from_reply_to_header;user_provided_email;phone_number;source;profession;email_receive_date')                
for i in range(messages, messages-N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject = decode_header(msg["Subject"])[0][0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode()

            if ("New Message From VPM's Almuni - Registration:" in subject) or ("New Message From VPM's Alumni - Registration:" in subject):                
                # email sender
                from_ = msg.get("From")
                from_name = from_.split("<mail@vpms.club>")
                From = from_name[0].replace('"', "").strip()

                received_spf = msg.get("Received-SPF")
                ip = re.findall(r'client-ip=[0-9]+(?:\.[0-9]+)+', received_spf)
                ip = ip[0].split("=")[1].strip()

                reply_to = msg.get("Reply-To")
                reply_to_email = re.findall(r'<(.*)>',reply_to)[0].strip()
                
                email_receive_date = msg.get("Date").strip()
                
                body_content_list = []

                # if the email message is multipart
                if msg.is_multipart():
                    # iterate over email parts
                    for part in msg.walk():
                        # extract content type of email
                        content_type = part.get_content_type()
                        content_disposition = str(part.get("Content-Disposition"))
                        try:
                            # get the email body
                            body = part.get_payload(decode=True).decode()
                        except:
                            pass
                        if content_type == "text/plain" and "attachment" not in content_disposition:
                            # print text/plain emails and skip attachments
                            print(body)
                        elif "attachment" in content_disposition:
                            # download attachment
                            filename = part.get_filename()
                            if filename:
                                if not os.path.isdir(subject):
                                    # make a folder for this email (named after the subject)
                                    os.mkdir(subject)
                                filepath = os.path.join(subject, filename)
                                # download attachment and save it
                                open(filepath, "wb").write(part.get_payload(decode=True))
                else:
                    # extract content type of email
                    content_type = msg.get_content_type()
                    # get the email body
                    body = msg.get_payload(decode=True).decode()
                    if content_type == "text/plain":
                        # print only text email parts
                        #print(body)
                        '''
                        for line in body.splitlines()[1:-1]:
                            #print(line)
                            body_content_list.append(line.split(" *: ")[1])
                        '''
                        for line in body.splitlines():
                            if " *: " in line:
                                body_content_list.append(line.split(" *: ")[1].strip())

                if(len(body_content_list) == 3):
                    print(f'{From};{ip};{reply_to_email};;{body_content_list[0]};{body_content_list[1]};{body_content_list[2]};{email_receive_date}')
                else:
                    print(f'{From};{ip};{reply_to_email};{body_content_list[0]};{body_content_list[1]};{body_content_list[2]};{body_content_list[3]};{email_receive_date}')

                body_content_list = []
                if content_type == "text/html":
                    # if it's HTML, create a new HTML file and open it in browser
                    if not os.path.isdir(subject):
                        # make a folder for this email (named after the subject)
                        os.mkdir(subject)
                    filename = f"{subject[:50]}.html"
                    filepath = os.path.join(subject, filename)
                    # write the file
                    open(filepath, "w").write(body)
                    # open in the default browser
                    webbrowser.open(filepath)
                #print("="*100)
imap.close()
imap.logout()

Total Messages:  328
from;user_ip_address;user_email_from_reply_to_header;user_provided_email;phone_number;source;profession;email_receive_date
Raju Kadam;192.185.143.5;raju.kadam@gmail.com;raju.kadam@gmail.com;7077977160;Pongbot;Alumni;Tue, 16 Jun 2020 18:07:54 +0000
Sham Naik;192.185.143.5;sham.naik@gmail.com;sham.naik@gmail.com;123456789;Sony;Alumni;Tue, 16 Jun 2020 17:54:12 +0000
Anand Marathe;192.185.201.2;anandamarathe@yahoo.co.in;anandamarathe@yahoo.co.in;8152748588;Thermo Fisher Scientific;Alumni;Tue, 16 Jun 2020 17:22:23 +0000
Anand Marathe;192.185.193.12;anandamarathe@yahoo.co.in;anandamarathe@yahoo.co.in;8152748588;Thermo Fisher Scientific;Alumni;Tue, 16 Jun 2020 16:47:24 +0000
sandeep Bhoot;192.185.196.18;sbhoot@yahoo.com;sbhoot@yahoo.com;14083103147;Pune University;Alumni;Tue, 16 Jun 2020 16:44:57 +0000
Vikram Gokhale;192.185.47.163;gokhalevikram@gmail.com;gokhalevikram@gmail.com;9193089827;VPM;Alumni;Tue, 16 Jun 2020 16:22:34 +0000
Atharva Kadam;192.185.49.40;atharvakadam

('BYE', [b'IMAP4rev1 Server logging out'])

In [ ]:
import re
reply_to = '''
"Mrs. Rajashree Umesh Patil\"" <rupatil@vpmthane.org>
'''
reply_to_email = re.findall(r'<(.*)>',reply_to)[0]
print(reply_to_email)

from_ = '''"Mrs. Rajashree Umesh Patil" <mail@vpms.club>
'''
from_name = from_.split("<mail@vpms.club>")
print(from_name[0].replace('"', "").strip())

In [ ]:
lines = '''I would like to register for the seminar
email *: spatils@gmail.com
Contact Number *: 9820211862
Name of the Institute / Organisation *: TCS
Designation *: Others
'''
for line in lines.splitlines()[1:]:
    print(line.split(" *: ")[1])

In [ ]:
import sys
!{sys.executable} -m pip install pandas

In [10]:
import pandas as pd
vp = pd.read_csv("v11_seminar_participants.csv", sep=";")
vp.sort_values("from")
vp = vp.drop_duplicates(subset="from")
# Get only participants who have emails
vp = vp.dropna(subset=["user_email_from_reply_to_header"])
vp

,from,user_ip_address,user_email_from_reply_to_header,user_provided_email,phone_number,source,profession,email_receive_date
0,Raju Kadam,192.185.143.5,raju.kadam@gmail.com,raju.kadam@gmail.com,7077977160,Pongbot,Alumni,"Tue, 16 Jun 2020 18:07:54 +0000"
1,Sham Naik,192.185.143.5,sham.naik@gmail.com,sham.naik@gmail.com,123456789,Sony,Alumni,"Tue, 16 Jun 2020 17:54:12 +0000"
2,Anand Marathe,192.185.201.2,anandamarathe@yahoo.co.in,anandamarathe@yahoo.co.in,8152748588,Thermo Fisher Scientific,Alumni,"Tue, 16 Jun 2020 17:22:23 +0000"
4,sandeep Bhoot,192.185.196.18,sbhoot@yahoo.com,sbhoot@yahoo.com,14083103147,Pune University,Alumni,"Tue, 16 Jun 2020 16:44:57 +0000"
5,Vikram Gokhale,192.185.47.163,gokhalevikram@gmail.com,gokhalevikram@gmail.com,9193089827,VPM,Alumni,"Tue, 16 Jun 2020 16:22:34 +0000"
...,...,...,...,...,...,...,...,...
294,Chetan kaluram harad,192.185.48.251,Chetanharad896@gmail.com,NaN,9594217920,Shivajirao. S. Jondhale polytechnic Asangaon,Student,"Wed, 3 Jun 2020 07:10:07 +0000"
295,Hrishikesh Nadkarni,192.185.51.35,dawjvoyager@gmail.com,NaN,9653335931,Vidya Prasarak Mandal's Polytechnic,Student,"Wed, 3 Jun 2020 07:07:21 +0000"
296,Supriya Bharat Suryarao,192.185.146.78,supriyasuryrao@gmail.com,NaN,9594214542,V. P. M's Polytechnic thane,Student,"Wed, 3 Jun 2020 07:05:01 +0000"
297,Shivam Yogesh Mishra,192.185.145.119,shvmyogi8433@gmail.com,NaN,8433579037,VPM'S Polytechnic,Student,"Wed, 3 Jun 2020 07:04:15 +0000"


In [11]:
# Export all emails to csv so that we can send them email using Mailgun API!
header = ["user_email_from_reply_to_header"]
vp.to_csv("v11_participants_email.csv", columns = header, index=False, header=False)

In [12]:
# Get unique professions
profession_list = sorted(vp.profession.unique())
print(*profession_list, sep="\n")

Alumni
Others
Professor
Student


In [13]:
# Get all students
students = vp[vp['profession'] == "Student"]
print(f"Total Students: {len(students)}")
students

Total Students: 183


,from,user_ip_address,user_email_from_reply_to_header,user_provided_email,phone_number,source,profession,email_receive_date
6,Atharva Kadam,192.185.49.40,atharvakadam@gmail.com,atharvakadam@gmail.com,3477252183,Stony Brook University,Student,"Mon, 15 Jun 2020 21:55:11 +0000"
7,Divya Sunil Chaudhari,192.185.46.187,divyachaudhari02019@gmail.com,divyachaudhari02019@gmail.com,9307973974,Government polytechnic Jalgao,Student,"Mon, 15 Jun 2020 09:28:13 +0000"
8,SNEHA PRASAD,192.185.45.147,prasad14sneha@gmail.com,prasad14sneha@gmail.com,8618378077,SIR M VISVESVARAYA INSTSTUTE OF TECHNOLOGY,Student,"Mon, 15 Jun 2020 06:19:31 +0000"
9,Deepak pandey,192.185.145.114,deepakp9454@gmail.com,deepakp9454@gmail.com,8356932415,S. H. JONDHALE POLYTECHNIC,Student,"Mon, 15 Jun 2020 04:33:11 +0000"
11,Avik Kadakia,192.185.51.61,avikkadakia@gmail.com,avikkadakia@gmail.com,5164234624,Stony Brook University,Student,"Mon, 15 Jun 2020 01:14:59 +0000"
...,...,...,...,...,...,...,...,...
294,Chetan kaluram harad,192.185.48.251,Chetanharad896@gmail.com,NaN,9594217920,Shivajirao. S. Jondhale polytechnic Asangaon,Student,"Wed, 3 Jun 2020 07:10:07 +0000"
295,Hrishikesh Nadkarni,192.185.51.35,dawjvoyager@gmail.com,NaN,9653335931,Vidya Prasarak Mandal's Polytechnic,Student,"Wed, 3 Jun 2020 07:07:21 +0000"
296,Supriya Bharat Suryarao,192.185.146.78,supriyasuryrao@gmail.com,NaN,9594214542,V. P. M's Polytechnic thane,Student,"Wed, 3 Jun 2020 07:05:01 +0000"
297,Shivam Yogesh Mishra,192.185.145.119,shvmyogi8433@gmail.com,NaN,8433579037,VPM'S Polytechnic,Student,"Wed, 3 Jun 2020 07:04:15 +0000"


In [14]:
# Get all professors
professors = vp[vp["profession"] == "Professor"]
print(f"Total Professors: {len(professors)}")
professors

Total Professors: 40


,from,user_ip_address,user_email_from_reply_to_header,user_provided_email,phone_number,source,profession,email_receive_date
12,Mrs Rama Ranjankar,192.185.145.187,ramaranjankar@rediffmail.com,ramaranjankar@rediffmail.com,9870774001,Pravin Patil College of Diploma engineering an...,Professor,"Sun, 14 Jun 2020 11:44:33 +0000"
32,Kavita Sanjay Singh,192.185.45.147,kavitassingh82@gmail.com,kavitassingh82@gmail.com,9769118547,Thakur Polytechnic,Professor,"Wed, 10 Jun 2020 14:36:40 +0000"
77,Minal Rahate,192.185.149.72,minal.rahate25@gmail.com,minal.rahate25@gmail.com,8087301208,Rajendra Mane Polytechnic Ambav Devrukh,Professor,"Mon, 8 Jun 2020 05:27:32 +0000"
87,Snehal Patil,192.185.50.84,snehalp63@gmail.com,snehalp63@gmail.com,8097680783,"S. H. Jondhale Polytechnic, Dombivli w",Professor,"Mon, 8 Jun 2020 04:27:34 +0000"
91,Sunil Nagare,192.185.149.77,sunilnagare04@gmail.com,sunilnagare04@gmail.com,9619015411,"VPM's Polytechnic, Thane",Professor,"Mon, 8 Jun 2020 04:04:18 +0000"
95,Dr.D.K.Nayak,192.185.48.251,dknayak@vpmthane.org,dknayak@vpmthane.org,9029823191,V.P.M's Polytechnic Thane,Professor,"Mon, 8 Jun 2020 03:02:18 +0000"
122,Dr.Kirti Agashe,192.185.45.89,ksagashe@gmail.com,ksagashe@gmail.com,9766628515,"VPM's Polytechnic,Thane",Professor,"Sun, 7 Jun 2020 11:57:58 +0000"
123,Vaishali Anupam Joshi,192.185.146.78,vajoshi@vpmthane.org,vajoshi@vpmthane.org,9821263821,"VPM's Polytechnic, Thane",Professor,"Sun, 7 Jun 2020 08:56:56 +0000"
133,Mrs. Rajashree Umesh Patil,192.185.148.104,rupatil@vpmthane.org,rupatil@vpmthane.org,9870115358,VPMs Polytechnic Thane,Professor,"Sat, 6 Jun 2020 14:11:30 +0000"
138,Mrs. Poonam Amit Kamble,192.185.145.108,pornimars11@gmaol.com,pornimars11@gmaol.com,8605141165,V.P.M's Polytechnic Thane,Professor,"Sat, 6 Jun 2020 07:31:46 +0000"


In [15]:
# Get all Alumni
alumni = vp[vp['profession'] == "Alumni"]
print(f"Total Alumni: {len(alumni)}")
alumni

Total Alumni: 10


,from,user_ip_address,user_email_from_reply_to_header,user_provided_email,phone_number,source,profession,email_receive_date
0,Raju Kadam,192.185.143.5,raju.kadam@gmail.com,raju.kadam@gmail.com,7077977160,Pongbot,Alumni,"Tue, 16 Jun 2020 18:07:54 +0000"
1,Sham Naik,192.185.143.5,sham.naik@gmail.com,sham.naik@gmail.com,123456789,Sony,Alumni,"Tue, 16 Jun 2020 17:54:12 +0000"
2,Anand Marathe,192.185.201.2,anandamarathe@yahoo.co.in,anandamarathe@yahoo.co.in,8152748588,Thermo Fisher Scientific,Alumni,"Tue, 16 Jun 2020 17:22:23 +0000"
4,sandeep Bhoot,192.185.196.18,sbhoot@yahoo.com,sbhoot@yahoo.com,14083103147,Pune University,Alumni,"Tue, 16 Jun 2020 16:44:57 +0000"
5,Vikram Gokhale,192.185.47.163,gokhalevikram@gmail.com,gokhalevikram@gmail.com,9193089827,VPM,Alumni,"Tue, 16 Jun 2020 16:22:34 +0000"
25,Dikshant Javheri,192.185.46.225,dikshantjaveri1@gmail.com,dikshantjaveri1@gmail.com,8828462695,CPA Solutions Pvt. Ltd.,Alumni,"Thu, 11 Jun 2020 05:41:28 +0000"
26,Pankaj Desai,192.185.53.25,desaip099@gmail.com,desaip099@gmail.com,8286832659,eternis fine chemicals Ltd,Alumni,"Thu, 11 Jun 2020 04:10:34 +0000"
27,Harshad Gadamale,192.185.146.87,hgadamale@gmail.com,hgadamale@gmail.com,418060455,Vpms polytechnic,Alumni,"Thu, 11 Jun 2020 03:53:31 +0000"
104,Varsha Arun Lohakare,192.185.54.2,varshalohakare0805@gmail.com,varshalohakare0805@gmail.com,9821507317,VPMs Polytechnic,Alumni,"Sun, 7 Jun 2020 15:35:06 +0000"
117,Saud Sayed,192.185.143.36,saudsayed46@gmail.com,saudsayed46@gmail.com,7977607375,"V. P. M, s",Alumni,"Sun, 7 Jun 2020 14:32:47 +0000"


In [16]:
# Get all Others
others = vp[vp['profession'] == "Others"]
print(f"Total Others: {len(others)}")
others

Total Others: 19


,from,user_ip_address,user_email_from_reply_to_header,user_provided_email,phone_number,source,profession,email_receive_date
10,SATISH MUTHE,192.185.144.97,udaymuthe@gmail.com,udaymuthe@gmail.com,9370290831,Sadguru Enterprise,Others,"Mon, 15 Jun 2020 03:52:16 +0000"
18,Kiran Ramchandra Dedhmukh,192.185.198.13,kiran.d@sparkelectro.com,kiran.d@sparkelectro.com,9820184682,Spark Electrosystems,Others,"Fri, 12 Jun 2020 04:59:04 +0000"
22,Narendra Digambar Apte,192.185.146.82,narendraapte@gmail.com,narendraapte@gmail.com,9967530747,NDA Consultancy & Engineering Services - Propr...,Others,"Thu, 11 Jun 2020 06:51:40 +0000"
24,Monoj Baruah,192.185.51.36,Monoj.Baruah@gmail.com,Monoj.Baruah@gmail.com,9845945826,Wipro,Others,"Thu, 11 Jun 2020 06:12:33 +0000"
54,Sunayana S Kalekar,192.185.49.184,Subodh.kalekar9@gmail.com,Subodh.kalekar9@gmail.com,9423982582,KALTRO ENTERPRISES,Others,"Tue, 9 Jun 2020 14:45:46 +0000"
56,SANJAY C. BHORE,192.185.51.36,scbhore@vpmthane.org,scbhore@vpmthane.org,25374494,VPMsPOLYTECHNIC,Others,"Tue, 9 Jun 2020 02:18:48 +0000"
57,Jyoti Babarao Kawale,50.116.127.1,jkawale16@gmail.com,jkawale16@gmail.com,9561513683,"Navjeevan Education Society Polytechnic, Bhand...",Others,"Mon, 8 Jun 2020 17:56:42 +0000"
59,Jitendra G Dongre,192.185.145.123,jitendra.d69@gmail.com,jitendra.d69@gmail.com,9969139790,Sardar Patel Institute of Technology,Others,"Mon, 8 Jun 2020 14:06:53 +0000"
62,Yogini Eknath Belhekar,50.116.125.1,mail2yogini@gmail.com,mail2yogini@gmail.com,9967129532,VPM's Polytechnic,Others,"Mon, 8 Jun 2020 12:38:22 +0000"
132,Sameer Patil,192.185.50.45,spatils@gmail.com,spatils@gmail.com,9820211862,TCS,Others,"Sat, 6 Jun 2020 14:33:45 +0000"


In [ ]:
# Get all unique colleges
#https://www.dummies.com/programming/python/printing-lists-using-python/
unique_college_list = sorted(vp.source.unique())
print(*unique_college_list, sep="\n")

In [ ]:
# Unique college list sorted
unique_college_string='''
Civil
Government Polytechnic
K.J.Somaiya Polytechnic
Kala Vidya Mandir Institute of Technology
M.I.T. Polytechnic
Mafer's MIT Polytechnic
MHF Polytechnic, Sanagmner
Pravin Patil Polytechnic College
Rajendra Mane Polytechnic
Shri Bhaghubhai Mafatlal Polytechnic
Samarth Polytechnic, Belhe
Sanjay Ghodawat Polytechnic
Shivajirao S. Jondhale Polytechnic
Soty
Thakur Polytechnic
VPM's Polytechnic
Sanjay Gandhi Polytechnic
'''

'''
SHRI BHAGHUBHAI MAFATLAL POLYTECHNIC
Government Polytechnic
Kala Vidya Mandir Institute of Polytechnic
M.I.T Polytechnic
Maeer'S MIT Polytechnic 
MHF Polytechnic
Shri Bhaghubhai Mafatlal Polytechnic
Sanjay Gandhi Polytechnic
Shivajirao S. Jhondhle Polytechnic
Samarth Polytechnic
Sanjay Ghodawat Polytechnic
Thakur Polytechnic
VPM's Polytechnic
Viva College of Diploma Engineering & Technology
'''

def get_unique_list(unique_string):    
    unique_list = []
    for line in unique_string.splitlines():
        if line:
            unique_list.append(line.strip())

    unique_list = sorted(unique_list)
    return unique_list

unique_colleges = get_unique_list(unique_college_string)
print(*unique_colleges, sep="\n")
print()
print(f"Students are participating from {len(unique_colleges)} colleges")
print(f"Students are participating from 6 Government Polytechnics")
print(f"So total colleges participating are 22")
print()

In [ ]:
# Unique cities where these colleges are located
unique_city_string = '''
Mumbai
Nandurbar
Ambad
Jalagon
Thane
Vikramgad
Ambav
Atigre
Asangaon
Belhe
Virar
Bellary
Pune
'''

'''
Nandurbar
Ambad
Jalgaon
Thane
Vikramgad
Pune
Sangamner
Asangaon
Atigre
Belhe
Virar
Bellary
'''

unique_cities = get_unique_list(unique_city_string)
print(*unique_cities, sep="\n")
print()
print(f"Students are participating from {len(unique_cities)} cities")
print()

In [ ]:
# https://www.thepythoncode.com/article/reading-emails-in-python
import imaplib
import email
from email.header import decode_header
import webbrowser
import os

# account credentials
username="quiz@vpms.club"
password="12Pappu34"
imap = imaplib.IMAP4_SSL("imappro.zoho.com")
# authenticate
imap.login(username, password)

status, messages = imap.select("INBOX")
# number of top emails to fetch
N = 3
# total number of emails
messages = int(messages[0])

for i in range(messages, messages-N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject = decode_header(msg["Subject"])[0][0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode()
            # email sender
            from_ = msg.get("From")
            print("Subject:", subject)
            print("From:", from_)

            print(msg)
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        print(body)
                    elif "attachment" in content_disposition:
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            if not os.path.isdir(subject):
                                # make a folder for this email (named after the subject)
                                os.mkdir(subject)
                            filepath = os.path.join(subject, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                if content_type == "text/plain":
                    # print only text email parts
                    print(body)
            if content_type == "text/html":
                # if it's HTML, create a new HTML file and open it in browser
                if not os.path.isdir(subject):
                    # make a folder for this email (named after the subject)
                    os.mkdir(subject)
                filename = f"{subject[:50]}.html"
                filepath = os.path.join(subject, filename)
                # write the file
                open(filepath, "w").write(body)
                # open in the default browser
                webbrowser.open(filepath)
            print("="*100)
imap.close()
imap.logout()